<a href="https://colab.research.google.com/github/maytlim/doh_data/blob/main/vacancies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!curl -s -O https://raw.githubusercontent.com/maytlim/doh_data/main/landline.py

In [2]:
import requests
import pandas as pd
import datetime
import landline
import gdown

In [3]:
  pd.options.display.float_format = '{:,.0f}'.format

In [4]:
try:
  import pdfx
except ModuleNotFoundError:
  !pip -q install pdfx
  import pdfx

     |████████████████████████████████| 5.6MB 7.6MB/s 
     |████████████████████████████████| 184kB 33.7MB/s 
     |████████████████████████████████| 3.2MB 37.7MB/s 
ERROR: requests 2.23.0 has requirement chardet<4,>=3.0.2, but you'll have chardet 4.0.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [5]:
!pip -q install xlsxwriter

     |████████████████████████████████| 153kB 8.3MB/s 


In [6]:
#Code from https://gist.github.com/korakot/51a917e1f53891d53be223439b0f75c1
from google.colab import auth
auth.authenticate_user()  # must authenticate

'''list all ids of files directly under folder folder_id'''
def folder_list(folder_id):
  from googleapiclient.discovery import build
  gdrive = build('drive', 'v3', cache_discovery=False).files()
  #gdrive = build('drive', 'v3').files()
  res = gdrive.list(q="'%s' in parents" % folder_id).execute()
  return [f['id'] for f in res['files']]

'''download all files from a gdrive folder to current directory'''
def folder_download(folder_id):
  for fid in folder_list(folder_id):
    !gdown -q --id $fid

In [7]:
ddrop_readme = requests.get('http://bit.ly/DataDropPH').url
ddrop_readme = ddrop_readme.split('/')[-1].split('?')[0]
#print(ddrop_readme)
folder_download(ddrop_readme)

In [8]:
readme_pdf = !ls *.pdf
readme_pdf = readme_pdf[0][1:-1]
readme_pdf
pdfx_tmp = pdfx.PDFx(readme_pdf)
ddlinks = pdfx_tmp.get_references_as_dict()

In [9]:
month_day = readme_pdf.split('_')
month = int(month_day[0][-2:])
day = int(month_day[1][:2])
month, day
year = 2021
print('DOH DataDrop Release:', year, month, day)

DOH DataDrop Release: 2021 4 18


In [10]:
downloaded = False
for ddlink in ddlinks['url']:
  if downloaded == False:
    if ddlink[:15] == 'https://bit.ly/' and ddlink[-3:] != 'ive':
      datadrop = requests.get(ddlink).url
      datadrop = datadrop.split('/')[-1].split('?')[0]
      folder_download(datadrop)
      print('Downloading DOH Data Drop...')
      downloaded = True
  else:
    continue

In [11]:
data_date = datetime.datetime(year, month, day)
#data_dir ='DOH COVID Data Drop_'  + data_date.strftime('%Y%m%d') + '/'

In [12]:
landline_masterlist = landline.make_masterlist()

In [13]:
# hospital daily patient census
cap_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 05 DOH Data Collect - Daily Report.csv'
# hospital weekly inventory of supplies
sup_filename = 'DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') +' - 06 DOH Data Collect - Weekly Report.csv'
print('weekly supplies inventory:', sup_filename)
print('daily capability count:', cap_filename)
hosp_code = 'hfhudcode' 
#hosp_name = 'cfname'

weekly supplies inventory: DOH COVID Data Drop_ 20210418 - 06 DOH Data Collect - Weekly Report.csv
daily capability count: DOH COVID Data Drop_ 20210418 - 05 DOH Data Collect - Daily Report.csv


In [14]:
#load capacity data
df_cap = pd.read_csv(cap_filename)
df_cap['cap_reportdate'] = pd.to_datetime(df_cap['reportdate'])
df_cap.drop(['updateddate', 'addeddate', 'reportdate', 'id', 'north_coord', 'east_coord'], axis=1, inplace=True)
df_cap[hosp_code] = df_cap[hosp_code].apply(lambda x: x.strip())

In [15]:
!gdown -q 'https://raw.githubusercontent.com/maytlim/doh_data/main/nhfr.json'

In [16]:
df_nhfr = pd.read_json('nhfr.json')
print('Refer to https://nhfr.doh.gov.ph/ for the latest data')

Refer to https://nhfr.doh.gov.ph/ for the latest data


In [17]:
df_cap = df_cap.merge(how='left', right=df_nhfr, left_on='hfhudcode', right_on='Health Facility Code')

In [18]:
#load supplies data
#df_sup = pd.read_csv(sup_filename)
#df_sup['sup_reportdate'] = pd.to_datetime(df_sup['reportdate'])
#df_sup.drop(['updateddate', 'addeddate', 'reportdate', 'id'], axis=1, inplace=True)
#df_sup[hosp_code] = df_sup[hosp_code].apply(lambda x: x.strip())

In [19]:
# quarantine facility daily patient census
#quarantine_daily = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 08 Quarantine Facility Data - Daily Report.csv')
# quarantine facility weekly inventory of supplies
#quarantine_weekly = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 09 Quarantine Facility Data - Weekly Report.csv')

In [20]:
# HCW availability
#baseline_v3 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 10 DOH Data Collect v3 - Baseline.csv')
# mechvent, swabbing, ipc
#baseline_v4 = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 11 DOH Data Collect v4 - Baseline.csv')
# quarantine and isolation facilities
#ttmf = pd.read_csv('DOH COVID Data Drop_ ' + data_date.strftime('%Y%m%d') + ' - 12 DDC TTMF Tracker v1 - Baseline.csv')

In [21]:
#print('{0}\n{1}'.format(df_cap.keys(), df_sup.keys()))
#print('{0}\n{1}'.format(quarantine_weekly.keys(), quarantine_daily.keys()))

In [22]:
reg_lookup = {'NCR': 'NATIONAL CAPITAL REGION (NCR)', 'ARMM': 'AUTONOMOUS REGION IN MUSLIM MINDANAO (ARMM)', 
              'CAR': 'CORDILLERA ADMINISTRA TIVE REGION (CAR)', 'I': 'REGION I (ILOCOS REGION)', 
              'II': 'REGION II (CAGAYAN VALLEY)', 'III': 'REGION III (CENTRAL LUZON)', 
              'IV-A': 'REGION IV-A (CALABAR ZON)', 'IV-B': 'REGION IV-B (MIMAROPA)', 
              'V': 'REGION V (BICOL REGION)', 'VI': 'REGION VI (WESTERN VISAYAS)', 
              'VII': 'REGION VII (CENTRAL VISAYAS)', 'VIII': 'REGION VIII (EASTERN VISAYAS)', 
              'IX': 'REGION IX (ZAMBOANGA PENINSULA)', 'X': 'REGION X (NORTHERN MINDANAO)', 
              'XI': 'REGION XI (DAVAO REGION)', 'XII': 'REGION XII (SOCCSKSA RGEN)', 
              'XIII': 'REGION XIII (CARAGA)'}

In [23]:
#prefix = ['ics', 'resdoc', 'nurse', 'medtech', 'respthe', 'radtech', 'support']
#suffix = ['_total', '_er', '_icu', '_ward'] 
#triage = ['resdoc_triage', 'nurse_triage', 'support_triage']
#added = ['Augmen_total']

In [24]:
def day_ago_from(n, from_date):
  showdate = from_date - datetime.timedelta(hours=24)*n
  return showdate.strftime('%Y-%m-%d')

In [25]:
def show_filtered(df_cap, days_ago, region='NATIONAL CAPITAL REGION (NCR)'):
  filter = (df_cap.mechvent_v > 0) & \
    (df_cap.cap_reportdate == day_ago_from(days_ago, data_date)) & \
    (df_cap.region == region) & \
    (df_cap.icu_v > 0) & \
    (df_cap.icu_o > 0)
  df_vacant = df_cap[filter][['cap_reportdate', 'cfname', 'city_mun', 'icu_v', 'isolbed_v', 'mechvent_v', 'beds_ward_v', 
                            'icu_o', 'isolbed_o', 'mechvent_o', 'beds_ward_o', 
                            'Landline Number', 'Landline Number 2', 'Fax Number', 'Ownership Major Classification']]
  df_vacant.rename({'Landline Number': 'landline', 'Landline Number 2': 'landline2', 'Fax Number': 'fax', 'Ownership Major Classification':'ownership'}, axis=1, inplace=True)
  df_vacant['landline'] = df_vacant['landline'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['landline2'] = df_vacant['landline2'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['fax'] = df_vacant['fax'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant.sort_values(by='icu_v', ascending=False, inplace=True)
  df_vacant.reset_index(inplace=True)
  return df_vacant

In [26]:
show_filtered(df_cap, days_ago=3)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,1728,2021-04-15,JOSE R. REYES MEMORIAL MEDICAL CENTER,SANTA CRUZ,14,14,10,70,5,51,3,93,(02) 8711 9491,(02) 8711 9498,(02) 8732 1077,Government
1,336882,2021-04-15,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,14,215,53,0,36,235,7,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
2,139151,2021-04-15,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,7,3,7,4,3,8,5,12,(02) 8716 1853,None,(02) 8716 1853,Private
3,368774,2021-04-15,OSPITAL NG MAYNILA MEDICAL CENTER,MALATE,5,41,1,21,5,13,2,63,(02) 8524 6063,(02) 8524 6064,(02) 8521 8490,Government
4,232355,2021-04-15,EAST AVENUE MEDICAL CENTER,QUEZON CITY,5,40,20,0,55,188,61,0,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
5,99873,2021-04-15,RIZAL MEDICAL CENTER,CITY OF PASIG,5,14,17,0,13,101,5,0,(02) 8865 8400,(02) 8671 9616,(02) 8671 9616,Government
6,106234,2021-04-15,VALENZUELA MEDICAL CENTER,CITY OF VALENZUELA,4,1,2,11,3,1,5,42,(02) 8294 6711,None,(02) 8291 4259,Government
7,78530,2021-04-15,MANILA DOCTORS HOSPITAL,ERMITA,4,28,10,0,20,42,14,0,(02) 8558 0888,None,(02) 8524 7376,Private
8,169369,2021-04-15,TONDO MEDICAL CENTER,TONDO I / II,4,37,2,0,14,5,4,84,(02) 8522 1174,(02) 8522 9244,(02) 8252 8661,Government
9,186095,2021-04-15,UP-PHILIPPINE GENERAL HOSPITAL,ERMITA,4,21,10,15,24,85,56,120,(02) 8554 8450,(02) 8523 7123,(02) 8524 2211,Government


In [27]:
show_filtered(df_cap, days_ago=2)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,1729,2021-04-16,JOSE R. REYES MEMORIAL MEDICAL CENTER,SANTA CRUZ,11,13,10,68,8,52,3,95,(02) 8711 9491,(02) 8711 9498,(02) 8732 1077,Government
1,336883,2021-04-16,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,11,214,41,0,39,236,9,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
2,252745,2021-04-16,QUEZON CITY GENERAL HOSPITAL,QUEZON CITY,8,0,1,0,21,91,3,0,(02) 8426 1383,(02) 8426 1321,(02) 8920 7081,Government
3,139152,2021-04-16,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,7,3,6,3,3,8,7,13,(02) 8716 1853,None,(02) 8716 1853,Private
4,186096,2021-04-16,UP-PHILIPPINE GENERAL HOSPITAL,ERMITA,6,18,12,9,22,85,58,125,(02) 8554 8450,(02) 8523 7123,(02) 8524 2211,Government
5,232356,2021-04-16,EAST AVENUE MEDICAL CENTER,QUEZON CITY,6,32,34,0,54,182,60,0,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
6,78531,2021-04-16,MANILA DOCTORS HOSPITAL,ERMITA,5,19,10,0,19,51,14,0,(02) 8558 0888,None,(02) 8524 7376,Private
7,368775,2021-04-16,OSPITAL NG MAYNILA MEDICAL CENTER,MALATE,5,43,1,20,5,13,2,64,(02) 8524 6063,(02) 8524 6064,(02) 8521 8490,Government
8,225031,2021-04-16,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,4,0,5,0,7,30,10,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
9,371662,2021-04-16,ALLIED CARE EXPERTS (ACE) MEDICAL CENTER-QUEZO...,QUEZON CITY,3,0,2,0,3,2,0,27,(02) 8325 0725,(02) 8283 3993,(02) 8283 4003,Private


In [28]:
show_filtered(df_cap, days_ago=1)

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,336884,2021-04-17,DR. JOSE N. RODRIGUEZ MEMORIAL HOSPITAL,CALOOCAN CITY,15,230,44,0,35,220,6,0,(02) 8294 2571,(02) 8962 9885,(02) 8962 8209,Government
1,1730,2021-04-17,JOSE R. REYES MEMORIAL MEDICAL CENTER,SANTA CRUZ,11,1,10,65,8,64,3,98,(02) 8711 9491,(02) 8711 9498,(02) 8732 1077,Government
2,225032,2021-04-17,PASIG CITY GENERAL HOSPITAL,CITY OF PASIG,9,19,5,0,4,35,10,0,(02) 8273 7981,(02) 8643 3333,(02) 8642 7380,Government
3,341915,2021-04-17,LUNG CENTER OF THE PHILIPPINES,QUEZON CITY,7,3,30,17,58,7,32,35,(02) 8924 6101,(02) 8924 6111,(02) 8924 6101,Government
4,139153,2021-04-17,UNIVERSITY OF THE EAST RAMON MAGSAYSAY MEMORIA...,QUEZON CITY,7,0,6,7,3,11,7,9,(02) 8716 1853,None,(02) 8716 1853,Private
5,169371,2021-04-17,TONDO MEDICAL CENTER,TONDO I / II,5,25,3,0,13,8,3,93,(02) 8522 1174,(02) 8522 9244,(02) 8252 8661,Government
6,186097,2021-04-17,UP-PHILIPPINE GENERAL HOSPITAL,ERMITA,4,18,12,9,24,85,58,125,(02) 8554 8450,(02) 8523 7123,(02) 8524 2211,Government
7,232357,2021-04-17,EAST AVENUE MEDICAL CENTER,QUEZON CITY,4,12,35,0,56,190,41,0,(02) 8928 0611,(02) 8929 2031,(02) 8922 0195,Government
8,368776,2021-04-17,OSPITAL NG MAYNILA MEDICAL CENTER,MALATE,4,43,1,16,6,13,2,68,(02) 8524 6063,(02) 8524 6064,(02) 8521 8490,Government
9,106525,2021-04-17,PASAY CITY GENERAL HOSPITAL,PASAY CITY,4,7,4,4,4,13,0,20,(02) 8833 6022,(02) 8831 3285,(02) 8551 3735,Government


In [29]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['II'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership


In [30]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['IV-A'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,182911,2021-04-17,GENERAL EMILIO AGUINALDO MEMORIAL HOSPITAL,TRECE MARTIRES CITY (CAPITAL),4,10,10,12,1,0,0,7,+63 464190063,None,None,Government
1,362412,2021-04-17,DE LA SALLE UNIVERSITY MEDICAL CENTER,CITY OF DASMARIÑAS,4,21,7,0,4,38,16,0,(02) 8988 3100,+63 464818000,None,Private
2,247309,2021-04-17,"ST. FRANCES CABRINI MEDICAL CENTER, INC.",SANTO TOMAS,3,3,5,1,3,5,1,15,+63 487784811,None,+63 487784819,Private
3,132229,2021-04-17,MEDICAL CENTER WESTERN BATANGAS,BALAYAN,2,3,1,3,5,1,3,5,+63 434071103,None,+63 434071101,Private
4,36729,2021-04-17,UNIHEALTH-SOUTHWOODS HOSPITAL AND MEDICAL CENT...,CITY OF BIÑAN,1,18,15,9,4,3,0,6,+63 498616385,None,None,Private
5,160982,2021-04-17,CARMONA HOSPITAL AND MEDICAL CENTER INC.,CARMONA,1,2,2,0,2,18,1,0,+63 464300295,None,None,Private
6,262287,2021-04-17,"GLOBAL CARE MEDICAL CENTER OF CANLUBANG, INC.",CITY OF CALAMBA,1,1,2,14,4,0,1,17,+63 495205626,None,None,Private
7,307405,2021-04-17,"ST. JAMES HOSPITAL, INC.",CITY OF SANTA ROSA,1,0,3,0,3,4,0,0,+63 495341254,None,None,Private
8,329072,2021-04-17,DANIEL O. MERCADO MEDICAL CENTER,CITY OF TANAUAN,1,10,2,0,3,0,0,15,+63 437781810,+63 437780960,None,Private
9,329941,2021-04-17,BATANGAS MEDICAL CENTER,BATANGAS CITY (CAPITAL),1,11,29,0,10,13,1,43,+63 437408307,None,+63 437230165,Government


In [31]:
show_filtered(df_cap, days_ago=1, region=reg_lookup['III'])

,index,cap_reportdate,cfname,city_mun,icu_v,isolbed_v,mechvent_v,beds_ward_v,icu_o,isolbed_o,mechvent_o,beds_ward_o,landline,landline2,fax,ownership
0,255057,2021-04-17,BULACAN MEDICAL CENTER,CITY OF MALOLOS (CAPITAL),9,76,54,31,21,18,1,69,+63 447910630,None,None,Government
1,103058,2021-04-17,BATAAN GENERAL HOSPITAL AND MEDICAL CENTER,CITY OF BALANGA (CAPITAL),8,32,28,21,37,76,20,10,+63 472372269,None,None,Government
2,27195,2021-04-17,RAMOS GENERAL HOSPITAL,CITY OF TARLAC (CAPITAL),4,5,2,0,1,11,0,0,+63 459827074,None,None,Private
3,107683,2021-04-17,ALLIED CARE EXPERTS (ACE) MEDICAL CENTER ?BALI...,BALIUAG,3,6,1,0,3,24,3,0,+63 448161000,None,+63 448161000,Private
4,339306,2021-04-17,JAMES L. GORDON MEMORIAL HOSPITAL,OLONGAPO CITY,3,35,5,0,1,34,1,10,+63 476021229,+63 476021230,+63 476021228,Government
5,143186,2021-04-17,"CENTRAL LUZON DOCTORS? HOSPITAL, INC.",CITY OF TARLAC (CAPITAL),2,17,3,0,6,34,2,0,+63 459820806,None,None,Private
6,227345,2021-04-17,QUALIMED HEALTH NETWORK SJDM,CITY OF SAN JOSE DEL MONTE,2,12,2,0,4,18,4,0,+63 443070000,None,None,Private
7,8167,2021-04-17,DR. PAULINO J. GARCIA MEMORIAL RESEARCH & MEDI...,CABANATUAN CITY,1,6,4,34,16,46,14,52,+63 444638286,+63 444639937,None,Government
8,32402,2021-04-17,"SACRED HEART HOSPITAL OF MALOLOS, INC.",CITY OF MALOLOS (CAPITAL),1,5,4,0,5,12,4,0,+63 447940561,None,+63 447912911,Private
9,127892,2021-04-17,PREMIERE MEDICAL CENTER,CABANATUAN CITY,1,16,1,9,3,4,1,12,+63 444637845,None,None,Private


In [32]:
roi = reg_lookup['NCR']
#roi = reg_lookup['II']
#roi = reg_lookup['IV-A']
filter = (df_cap.cap_reportdate == day_ago_from(1, data_date)) & (df_cap.region == roi)
occupied = df_cap[filter]['icu_o'].sum()
vacant = df_cap[filter]['icu_v'].sum()
pct_used = (occupied / (vacant + occupied)) * 100

In [33]:
def dump_daily(df_cap, days_ago):
  filter = (df_cap.cap_reportdate == day_ago_from(days_ago, data_date)) 
  df_vacant = df_cap[filter][['cfname', 'region', 'province', 'city_mun', 
                              'icu_v', 'isolbed_v', 'mechvent_v', 'beds_ward_v', 
                              'icu_o', 'isolbed_o', 'mechvent_o', 'beds_ward_o', 
                              'Landline Number', 'Landline Number 2', 'Fax Number', 'Ownership Major Classification']]
  df_vacant.rename({'Landline Number': 'landline', 'Landline Number 2': 'landline2', 'Fax Number': 'fax', 'Ownership Major Classification':'ownership'}, axis=1, inplace=True)
  df_vacant['landline'] = df_vacant['landline'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['landline2'] = df_vacant['landline2'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant['fax'] = df_vacant['fax'].apply(lambda x: landline.convert_landline(x, landline_masterlist))
  df_vacant.sort_values(by='icu_v', ascending=False, inplace=True)
  df_vacant.reset_index(inplace=True, drop=True)
  return df_vacant

In [34]:
#https://stackoverflow.com/questions/29463274/simulate-autofit-column-in-xslxwriter
def get_col_widths(dataframe):
    # First we find the maximum length of the index column   
    idx_max = max([len(str(s)) for s in dataframe.index.values] + [len(str(dataframe.index.name))])
    # Then, we concatenate this to the max of the lengths of column name and its values for each column, left to right
    return [idx_max] + [max([len(str(s)) for s in dataframe[col].values] + [len(col)]) for col in dataframe.columns]

In [35]:
writer = pd.ExcelWriter('hospitals_DOHDataDrop_' + data_date.strftime('%Y%m%d') + '.xlsx', engine='xlsxwriter')
workbook  = writer.book
format_wrap = workbook.add_format({'text_wrap': True})

In [36]:
readme = 'README'
notes = []
notes.append('Data source: DOH Data Drop ' + data_date.strftime('%Y-%m-%d'))
notes.append('Data source link is in the README PDF @ ' + 'https://bit.ly/DataDropPH')
notes.append('Info here is dated -- call the ONE HOSPITAL COMMAND CENTER for current availability. Reach them through the ff. numbers: (02) 8865-0500, 0915-777-7777 and 0919-977-3333')
notes.append('cap_reportdate = date of report for icu, isolation rooms, beds in ward, ventilators')
notes.append('IMPORTANT: Please check https://nhfr.doh.gov.ph/rfacilities2list.php for latest hospital info')
notes.append('Default sorting is by number of vacant ICU beds - largest to smallest.')
notes.append('This is not an official document. It has not been validated - there could be errors.')
notes.append('Refer to ncovtracker.doh.gov.ph for official announcements')
notes.append('Very important: Please contact the ONE HOSPITAL COMMAND CENTER if you are looking for a hospital room / healthcare referral.')
notes.append('OHCC contact: (02) 8865-0500, 0915-777-7777 and 0919-977-3333')
notes.append('DOH COVID-19 toll-free hotline number: 1555')
notes.append('Alternatively, contact your LGU - https://www.rappler.com/nation/list-covid-19-hotline-numbers-one-hospital-command-metro-manila')
notes.append('Source code at https://github.com/maytlim/doh_data')
df_readme = pd.DataFrame(notes)
df_readme.to_excel(writer, readme)

In [37]:
for days_ago in range(1, 4):
  reportdate = day_ago_from(days_ago, data_date)
  df_dumpdaily = dump_daily(df_cap, days_ago)
  df_dumpdaily.to_excel(writer, reportdate)
  worksheet = writer.sheets[reportdate]
  worksheet.set_row(0, None, cell_format=format_wrap)
  for i, width in enumerate(get_col_widths(df_dumpdaily)):
    worksheet.set_column(i, i, min(20, max(width + 4, 5)))
  (max_row, max_col) = df_dumpdaily.shape
  # Set the autofilter
  worksheet.autofilter(0, 1, max_row, max_col)

In [38]:
writer.close()